# Description

It combines all coefficient values in one tissue (see `Settings` below) into a single dataframe for easier processing later.

This notebook incorporates results using MIC, which was computed only in a subset of gene pairs due to its computational complexity.

# Modules loading

In [ ]:
import pandas as pd

from clustermatch import conf
from clustermatch.utils import get_upper_triag

# Settings

In [ ]:
DATASET_CONFIG = conf.GTEX
# whole blood by default, but this is a parameters cells that can be changed when running papermill
GTEX_TISSUE = "whole_blood"
GENE_SEL_STRATEGY = "var_pc_log2"

METHOD_NAME = "mic"

# Paths

In [ ]:
COMPARISONS_DIR = DATASET_CONFIG["RESULTS_DIR"] / "comparison_others"
display(COMPARISONS_DIR)

In [ ]:
INPUT_DIR = COMPARISONS_DIR / METHOD_NAME
display(INPUT_DIR)

In [ ]:
INPUT_CORR_FILE_TEMPLATE = (
    DATASET_CONFIG["SIMILARITY_MATRICES_DIR"]
    / DATASET_CONFIG["SIMILARITY_MATRIX_FILENAME_TEMPLATE"]
)
display(INPUT_CORR_FILE_TEMPLATE)

In [ ]:
INPUT_CORR_FILE_TEMPLATE = (
    DATASET_CONFIG["SIMILARITY_MATRICES_DIR"]
    / DATASET_CONFIG["SIMILARITY_MATRIX_FILENAME_TEMPLATE"]
)
display(INPUT_CORR_FILE_TEMPLATE)

In [ ]:
OUTPUT_FILE = (
    COMPARISONS_DIR / f"gtex_v8_data_{GTEX_TISSUE}-{GENE_SEL_STRATEGY}-all.pkl"
)
display(OUTPUT_FILE)

# Load data

## Clustermatch

In [ ]:
clustermatch_df = pd.read_pickle(
    str(INPUT_CORR_FILE_TEMPLATE).format(
        tissue=GTEX_TISSUE,
        gene_sel_strategy=GENE_SEL_STRATEGY,
        corr_method="clustermatch",
    )
)

In [ ]:
clustermatch_df.shape

In [ ]:
clustermatch_df.head()

In [ ]:
clustermatch_df = get_upper_triag(clustermatch_df)

In [ ]:
clustermatch_df = (
    clustermatch_df.unstack()
    .rename_axis((None, None))
    .dropna()
    .sort_index()
    .rename("clustermatch")
)

In [ ]:
clustermatch_df.shape

In [ ]:
clustermatch_df.head()

## Pearson

In [ ]:
pearson_df = pd.read_pickle(
    str(INPUT_CORR_FILE_TEMPLATE).format(
        tissue=GTEX_TISSUE,
        gene_sel_strategy=GENE_SEL_STRATEGY,
        corr_method="pearson",
    )
)

In [ ]:
pearson_df.shape

In [ ]:
pearson_df.head()

In [ ]:
pearson_df = get_upper_triag(pearson_df)

In [ ]:
# make pearson abs
pearson_df = (
    pearson_df.unstack()
    .rename_axis((None, None))
    .dropna()
    .abs()
    .sort_index()
    .rename("pearson")
)

In [ ]:
pearson_df.shape

In [ ]:
pearson_df.head()

## Spearman

In [ ]:
spearman_df = pd.read_pickle(
    str(INPUT_CORR_FILE_TEMPLATE).format(
        tissue=GTEX_TISSUE,
        gene_sel_strategy=GENE_SEL_STRATEGY,
        corr_method="spearman",
    )
)

In [ ]:
spearman_df.shape

In [ ]:
spearman_df.head()

In [ ]:
spearman_df = get_upper_triag(spearman_df)

In [ ]:
# make spearman abs
spearman_df = (
    spearman_df.unstack()
    .rename_axis((None, None))
    .dropna()
    .abs()
    .sort_index()
    .rename("spearman")
)

In [ ]:
spearman_df.shape

In [ ]:
spearman_df.head()

## MIC

In [ ]:
mic_df = pd.read_pickle(
    str(INPUT_CORR_FILE_TEMPLATE).format(
        tissue=GTEX_TISSUE,
        gene_sel_strategy=GENE_SEL_STRATEGY,
        corr_method="mic_parallel",
    )
)

In [ ]:
mic_df.shape

In [ ]:
mic_df.head()

In [ ]:
mic_df = get_upper_triag(mic_df)

In [ ]:
mic_df = mic_df.unstack().rename_axis((None, None)).dropna().sort_index().rename("mic")

In [ ]:
mic_df.shape

In [ ]:
mic_df.head()

## Checks

In [ ]:
assert (
    len(set(clustermatch_df.index).intersection(set(mic_df.index)))
    == clustermatch_df.index.shape[0]
)

In [ ]:
assert (
    len(set(clustermatch_df.index).intersection(set(mic_df.index)))
    == mic_df.index.shape[0]
)

In [ ]:
assert (
    len(set(pearson_df.index).intersection(set(mic_df.index))) == mic_df.index.shape[0]
)

In [ ]:
assert (
    len(set(spearman_df.index).intersection(set(mic_df.index))) == mic_df.index.shape[0]
)

## Merge

In [ ]:
df = pd.concat(
    [clustermatch_df, pearson_df, spearman_df, mic_df], join="inner", axis=1
).sort_index()

In [ ]:
display(df.shape)
assert df.shape[0] == mic_df.shape[0]

In [ ]:
assert not df.isna().any().any()

In [ ]:
df.head()

# Save

In [ ]:
df.to_pickle(OUTPUT_FILE)